In [1]:
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import numpy as np
import cv2
import random

import importlib
import config

importlib.reload(config)
# from datasets import load_dataset


[nltk_data] Downloading package words to
[nltk_data]     C:\Users\kuzga\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\kuzga\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


<module 'config' from 'c:\\Users\\kuzga\\OneDrive\\Рабочий стол\\ShiftLab-2024-Computer-Vision\\config.py'>

In [2]:
image_size = 64

In [3]:
def cv2_show(img, key='q', time=0, window_name='cv2'):
    print(time)
    cv2.imshow(window_name, img)
    if time:
        cv2.waitKey(time)
        cv2.destroyAllWindows()
    else:
        printed_key = cv2.waitKey(0)
        if printed_key == key:
            cv2.destroyAllWindows()


def stack_images(images, direction='vertical'):
    if not images or any(img is None for img in images):
        raise ValueError(
            "Список изображений пуст или содержит недопустимые значения")

    if direction == 'horizontal':
        min_height = min(img.shape[0] for img in images)
        resized_images = [cv2.resize(img, (int(img.shape[1] * (min_height / img.shape[0])), min_height))
                          for img in images]
        stacked_image = np.hstack(resized_images)
    elif direction == 'vertical':
        max_width = max(img.shape[1] for img in images)
        resized_images = [cv2.resize(img, (max_width, int(img.shape[0] * (max_width / img.shape[1]))))
                          for img in images]
        stacked_image = np.vstack(resized_images)
    else:
        raise ValueError("direction должен быть 'horizontal' или 'vertical'")

    return stacked_image


def add_noise_and_distortion(img):
    noise = np.random.uniform(0, 50, img.shape).astype(np.uint8)
    noisy_img = cv2.add(img, noise)

    # искажения с помощью линий
    num_lines = np.random.randint(0, 10)
    for _ in range(num_lines):
        x1, y1 = np.random.randint(
            0, img.shape[1]), np.random.randint(0, img.shape[0])
        x2, y2 = np.random.randint(
            0, img.shape[1]), np.random.randint(0, img.shape[0])
        cv2.line(noisy_img, (x1, y1), (x2, y2), (0, 0, 0), 1)

    return noisy_img

In [4]:
def generate_text_image(text, img=None, noise=False, max_char_y_offset=0.025, max_char_x_offset=0.025):
    gen_height, gen_width = image_size, image_size * 4

    if img is None:
        background_color = np.random.randint(0, 256, size=3, dtype=np.uint8)
        img = np.ones((gen_height, gen_width, 3), dtype=np.uint8) * background_color

    font_path = random.choice(config.fonts_paths)
    font_size = random.randint(20, 35)
    font = ImageFont.truetype(font_path, font_size)

    pil_img = Image.fromarray(img)
    draw = ImageDraw.Draw(pil_img)

    # Находим позицию текста
    bbox = draw.textbbox((0, 0), text, font=font)
    text_size = (bbox[2] - bbox[0], bbox[3] - bbox[1])

    # Уменьшаем размер шрифта, если текст не помещается
    while text_size[0] > img.shape[1] or text_size[1] > img.shape[0]:
        font_size -= 1
        font = ImageFont.truetype(font_path, font_size)
        bbox = draw.textbbox((0, 0), text, font=font)
        text_size = (bbox[2] - bbox[0], bbox[3] - bbox[1])

    text_x = (img.shape[1] - text_size[0]) // 2
    text_y = (img.shape[0] - text_size[1]) // 2

    # Рисуем текст с небольшими случайными смещениями
    for char in text:
        bbox = draw.textbbox((text_x, text_y), char, font=font)
        char_width = bbox[2] - bbox[0]
        char_height = bbox[3] - bbox[1]

        x_offset = random.randint(int(-char_width * max_char_x_offset), int(char_width * max_char_x_offset))
        y_offset = random.randint(int(-char_height * max_char_y_offset), int(char_height * max_char_y_offset))
        draw.text((text_x + x_offset, text_y + y_offset), char, font=font, fill=0)  # Черный текст

        text_x += char_width  # Сдвигаем на ширину текущей буквы

    img = np.array(pil_img)  # Преобразуем обратно в формат OpenCV

    return add_noise_and_distortion(img) if noise else img

In [ ]:
images = []

for _ in range(10):
    text = random.choice(config.texts)
    result = generate_text_image(text, noise=True)
    # print(result.shape)
    images.append(result)

stacked_images = stack_images(images)
cv2_show(stacked_images, time=10000)

10000
